In [2]:
import os
import asyncio
import aiohttp
import pandas as pd
from sqlalchemy import create_engine
from tqdm import tqdm
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry

# Load environment variables
load_dotenv()
api_key = os.getenv('api')

# Rate limit parameters
RATE_LIMIT = 100  # 100 requests per minute
TIME_WINDOW = 60  # 60 seconds

# Database connection
engine = create_engine('mssql+pyodbc://LUCAS\\SQLEXPRESS01/Riotdb?driver=ODBC+Driver+17+for+SQL+Server')

# 1. Retrieve Summoner IDs and PUUIDs
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_puuid(session, summoner_id):
    api_puuid = f"https://br1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={api_key}"
    async with session.get(api_puuid) as response:
        response.raise_for_status()
        puuid_json = await response.json()
        return puuid_json.get('puuid', None)

async def fetch_all_puuids(session, summoner_ids):
    tasks = [fetch_puuid(session, summoner_id) for summoner_id in summoner_ids]
    return await asyncio.gather(*tasks)

# 2. Retrieve Match IDs
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_match_ids(session, puuid):
    api_url = f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key={api_key}"
    async with session.get(api_url) as response:
        response.raise_for_status()
        return await response.json()

@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_WINDOW)
async def fetch_match_detail(session, match_id):
    api_match_details = f"https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}"
    try:
        async with session.get(api_match_details) as response:
            response.raise_for_status()
            details_json = await response.json()
            participants = details_json.get('info', {}).get('participants', [])
            participants_df = pd.json_normalize(participants)
            participants_df.insert(0, 'Match_ID', match_id)
            return participants_df
    except aiohttp.ClientResponseError as e:
        print(f"HTTPError for match_id {match_id}: {e}")
        return None

# 3. Process a single PUUID and fetch match IDs and details
async def process_puuid(session, puuid):
    match_ids = await fetch_match_ids(session, puuid)
    match_details_list = []
    for match_id in match_ids:
        match_details_json = await fetch_match_detail(session, match_id)
        if match_details_json is not None:
            match_details_list.append(match_details_json)
    return match_details_list

async def fetch_all_match_details(session, puuids):
    match_details = []
    for puuid in puuids:
        match_details += await process_puuid(session, puuid)
    return match_details

async def main():
    # Step 1: Fetch Summoner IDs and PUUIDs
    liga = input("Insira a liga desejada: ").upper()
    api_liga = f"https://br1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{liga}/I?api_key={api_key}"

    async with aiohttp.ClientSession() as session:
        async with session.get(api_liga) as liga_resposta:
            liga_resposta.raise_for_status()
            liga_json = await liga_resposta.json()

    summoner_ids = [entry['summonerId'] for entry in liga_json[:100]]  # Fetch only 100 summoners
    df_liga = pd.DataFrame(summoner_ids, columns=['summonerId'])

    # Fetch PUUIDs asynchronously
    async with aiohttp.ClientSession() as session:
        puuids = await fetch_all_puuids(session, tqdm(df_liga['summonerId'], desc="Fetching PUUIDs", unit="summonerId"))
    
    # Add the PUUIDs to the DataFrame
    df_liga['puuid'] = puuids

    # Save the Summoner IDs and PUUIDs to the database
    df_liga.to_sql('SummID_PUUID', con=engine, if_exists='replace', index=False)
    print("Summoner IDs and PUUIDs saved successfully.")

    # Step 2: Fetch Match IDs and Match Details for each PUUID
    async with aiohttp.ClientSession() as session:
        match_details_list = await fetch_all_match_details(session, tqdm(puuids, desc="Fetching match details", unit="puuid"))

    # Concatenate match details into a DataFrame
    if match_details_list:
        df_match_details = pd.concat(match_details_list, ignore_index=True)
        df_match_details.to_sql('Detalhes_Liga', con=engine, if_exists='replace', index=False)
        print("Match details saved successfully.")
    else:
        print("No match details retrieved.")

if __name__ == "__main__":
    asyncio.run(main())


RuntimeError: asyncio.run() cannot be called from a running event loop